In [2]:
import os
import sys

join = os.path.join
dirname = os.path.dirname

sys.path.append('../')

In [3]:
import json
import config
import numpy as np
import pandas as pd

from tqdm.contrib import tzip
from copy import deepcopy as copy
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

/mnt/conda/envs/arsearch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from src.text_simlirity import Text_Similarity, CTR_CQR_Score, Simbert_Score

ts = Text_Similarity()
def extract_feature(query, document, title=None):
    if title is None:
        total_text_score, total_embed_score = ts.seprate_similarity(query, document)
        document = [s for p in document.split('\n') for s in p.split('。') if s!= '']
        scores = []
        related_q_d_num = 0
        for line in document:
            line += '。'
            text_score, embed_score = ts.seprate_similarity(query, line)
            scores.append([text_score, embed_score, text_score + embed_score])
            line_score = text_score + embed_score
            # max_q_d_score = max(max_q_d_score, line_score)
            if line_score >= 0.7:
                related_q_d_num += 1
        related_q_d_num_score = related_q_d_num / len(document)
        scores.sort(key=lambda x: x[-1], reverse=True)
        scores = scores[:1]
        features = [total_text_score, total_embed_score]
        if len(scores) < 1:
            scores = scores + (1-len(scores))*[[-1, -1, -2]]
        # print(scores)
        features = features + list(np.array(scores).flat) + [len(document)]
        # print(features)
        return features

In [11]:
RANDOM_STATE = 0

In [12]:
def jsonl_load(path):
    data = []
    lines = open(path).readlines()
    for line in lines:
        json_line = json.loads(line.strip('\n'))
        data.append(copy(json_line))
    return data

In [13]:
def process_data(data):
    postive_items = []
    titles = [item['summary'] for item in data]
    contents = [item['content'] for item in data]
    for t, c in zip(titles, contents):
        postive_items.append([t, c, 1])
    
    negative_items = []
    shuffle_titles = shuffle(titles, random_state=RANDOM_STATE)
    for t, c in zip(shuffle_titles, contents):
        negative_items.append([t, c, 0])
    
    dataset = postive_items + negative_items
    dataset = pd.DataFrame(dataset, columns=['query', 'contents', 'label'])
    X_train, X_test, y_train, y_test = train_test_split(dataset[['query', 'contents']], dataset['label'], test_size=0.25, random_state=RANDOM_STATE)
    return X_train, X_test, y_train, y_test
    

In [14]:
dev_data_path = join(config.PATH_ARGS.DATA_DIR, 'LCSTS_new', 'dev.json')
train_data_path = join(config.PATH_ARGS.DATA_DIR, 'LCSTS_new', 'train.json')
dev_data = jsonl_load(dev_data_path)
train_data = jsonl_load(train_data_path)
all_data = dev_data + train_data

X_train, X_test, y_train, y_test = process_data(all_data)

KeyboardInterrupt: 

In [ ]:
x_train_features = []
# f_train = open(join(config.PATH_ARGS.DATA_DIR, 'LCSTS_new', 'x_train_features.txt'), 'w')
f_train = open(join(config.PATH_ARGS.DATA_DIR, 'LCSTS_new', 'x_train_features.txt'), 'a')
for item, label in tzip(X_train.values[11834:], y_train.values[11834:], desc='x_train'):
    feature = extract_feature(item[0], item[1])
    # print(item[0], item[1], ' '.join([str(i) for i in copy(feature)]) + f' {label}\n')
    f_train.write(' '.join([str(i) for i in copy(feature)]) + f' {label}\n')
    f_train.flush()
    x_train_features.append(copy(feature))
    # print(' '.join([str(i) for i in feature]) + f' {label}\n')
    
x_train_features = pd.DataFrame(x_train_features)
x_train_features['label'] = y_train
x_train_features_path = join(config.PATH_ARGS.DATA_DIR, 'LCSTS_new', 'x_train_features_1.0.csv')
x_train_features.to_csv(x_train_features_path, index=False)

x_test_features = []
# f_test = open(join(config.PATH_ARGS.DATA_DIR, 'LCSTS_new', 'x_test_features.txt'), 'w')
f_test = open(join(config.PATH_ARGS.DATA_DIR, 'LCSTS_new', 'x_test_features.txt'), 'a')
for item, label in tzip(X_test.values, desc='x_test'):
    feature = extract_feature(item[0], item[1])
    x_test_features.append(feature)
    f_test.write(' '.join([str(i) for i in feature]) + f' {label}\n')
    f_test.flush()
    
x_test_features = pd.DataFrame(x_test_features)
x_test_features['label'] = y_test
x_test_features_path = join(config.PATH_ARGS.DATA_DIR, 'LCSTS_new', 'x_test_features_1.0.csv')
x_test_features.to_csv(x_test_features_path, index=False)
  

    

x_train:   0%|          | 0/2210318 [00:00<?, ?it/s]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.668 seconds.
Prefix dict has been built successfully.
x_train:   0%|          | 5385/2210318 [4:38:00<1897:12:30,  3.10s/it]


KeyboardInterrupt: 